In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report, accuracy_score
import kagglehub

In [ ]:
path = kagglehub.dataset_download("demartlectus/wildberries-search-queries-from-wildhack-2021")
file_path = f"{path}/query_popularity.csv"
data = pd.read_csv(file_path)
print(data.head())

                    query  query_popularity
0                 ноутбук                10
1  куртка женская осенняя                10
2         ботинки женские                10
3              видеокарта                10
4  пальто женское осеннее                10


In [ ]:
# Создание целевой переменной (категории популярнсоти)
def categorize_popularity(popularity):
    if popularity >= 8:
        return "Популярный запрос"
    elif popularity >= 5:
        return "Средне популярный запрос"
    else:
        return "Не популярный запрос"

data['popularity_category'] = data['query_popularity'].apply(categorize_popularity)
data['query'] = data['query'].fillna("")

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data['query'], data['popularity_category'], test_size=0.2, random_state=50)

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение классификатора
classifier = PassiveAggressiveClassifier()
classifier.fit(X_train_vec, y_train)

# Предсказание на тестовых данных
y_pred = classifier.predict(X_test_vec)

# Оценка качества модели
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Classification Report:
                          precision    recall  f1-score   support

    Не популярный запрос       0.58      0.59      0.59     27035
       Популярный запрос       0.48      0.59      0.53     20243
Средне популярный запрос       0.36      0.27      0.31     20120

                accuracy                           0.50     67398
               macro avg       0.47      0.48      0.47     67398
            weighted avg       0.48      0.50      0.49     67398

Accuracy: 0.49502952609869727


In [ ]:
external_data = pd.read_csv(file_path)
external_data['popularity_category'] = external_data['query_popularity'].apply(categorize_popularity)
external_data['query'] = external_data['query'].fillna("")

# Векторизация текста
X_external_vec = vectorizer.transform(external_data['query'])
y_external_pred = classifier.predict(X_external_vec)

# Оценка качества модели на внешних данных
print("External Data Classification Report:")
print(classification_report(external_data['popularity_category'], y_external_pred))
print(f"External Data Accuracy: {accuracy_score(external_data['popularity_category'], y_external_pred)}")

External Data Classification Report:
                          precision    recall  f1-score   support

    Не популярный запрос       0.59      0.61      0.60    134723
       Популярный запрос       0.49      0.61      0.54    101102
Средне популярный запрос       0.39      0.28      0.33    101163

                accuracy                           0.51    336988
               macro avg       0.49      0.50      0.49    336988
            weighted avg       0.50      0.51      0.50    336988

External Data Accuracy: 0.5094929196291856


In [ ]:
# Функция для предсказания категории запроса
def predict_query_category(query, vectorizer, classifier):
    # Очистка текста (заполнение пропущенных значений, если есть)
    query = query if isinstance(query, str) else ""

    # Преобразование текста в числовой формат
    query_vec = vectorizer.transform([query])

    # Предсказание категории
    predicted_category = classifier.predict(query_vec)

    return predicted_category[0]

# Пример использования
user_query = input("Введите ваш запрос: ")  # Пользователь вводит запрос
predicted_category = predict_query_category(user_query, vectorizer, classifier)
print(f"Запрос '{user_query}' относится к категории: {predicted_category}")